In [ ]:
import sys
import os
import pandas as pd
import csv
import argparse
import gtools

In [ ]:
parser = argparse.ArgumentParser(description='''
Filters a duplex-seq variant tsv that has been generated by add_duplex_filter_columns.py. The output may still contain germline mutations 
and contaminant errors which will need to be filtered out by removing variants present in multiple samples. The letter code of each filter 
is the same as in Figure S1. Filters are applied in alphabetical order.
    ''', formatter_class=gtools.EscapeFormatter)
parser.add_argument('-i', '--input', dest='input', metavar='TSV', type=str,
                   help='generated by running add_duplex_filter_columns.py on the output of duplex_caller.py (default: stdin)')
parser.add_argument('-o', '--output', dest='output', metavar='TSV', type=str,
                   help='filtered output tsv. Will just be a subset of the input rows (default: stdout)')
parser.add_argument('-A', '--min_frac_init', dest='min_frac_init', metavar='FLOAT', type=float, default=0.24,
                   help='initial minimum fraction of supporting reads per strand (default: 0.24)')
parser.add_argument('-B', '--min_strand_cov', dest='min_strand_cov', metavar='INT', type=int, default=2,
                   help='minimum reads per strand overlapping variant (default: 2)')
parser.add_argument('-C', '--min_total_cov', dest='min_total_cov', metavar='INT', type=int, default=6,
                   help='minimum total reads overalpping variant (default: 6)')
parser.add_argument('-D', '--min_high_bq', dest='min_high_bq', metavar='INT', type=int, default=4,
                   help='minimum supporting reads with high BQ (>=30) (default: 4)')
parser.add_argument('-E', '--min_avg_mq', dest='min_avg_mq', metavar='FLOAT', type=float, default=10,
                   help='minimum average MQ of supporting reads (default: 10)')
parser.add_argument('-F', '--max_end_mismatch', dest='max_end_mismatch', metavar='FLOAT', type=float, default=0.21,
                   help='maximum mismatch rate between a position and a fragment end before clipping (default: 0.21)')
parser.add_argument('-G', '--min_indel_end_dist', dest='min_indel_end_dist', metavar='INT', type=int, default=6,
                   help='minimum distance between indels and a fragment end (default: 6)')
parser.add_argument('-H', '--min_cov_percentile', dest='min_cov_percentile', metavar='FLOAT', type=float, default=0.007,
                   help='minimum sequencing coverage percentile of the variant position (default: 0.007, i.e. 0.7 percentile)')
parser.add_argument('-I', '--max_poly_at', dest='max_poly_at', metavar='INT', type=int, default=7,
                   help='maximum length of a poly-A/T repeat ending at the variant position (default: 7)')
parser.add_argument('-J', '--max_poly_rep', dest='max_poly_rep', metavar='INT', type=int, default=4,
                   help='maximum length of a poly-di/trinucleotide repeat end at the variant position (default: 4)')
parser.add_argument('-K', '--max_control_sup', dest='max_control_sup', metavar='INT', type=int, default=6,
                   help='maximum support for the variant across the control samples (default: 6)')
parser.add_argument('-L', '--max_frag_len', dest='max_frag_len', metavar='INT', type=int, default=300,
                   help='maximum fragment length (default: 300)')
parser.add_argument('-M', '--max_contamination_dist', dest='max_contamination_dist', metavar='INT', type=int, default=4,
                   help='maximum distance between two variants passing prior filters (A-L) within a single fragment \
                    (default: 4, i.e. there can be at most 3 bases between passing variants)')
parser.add_argument('-N', '--min_frac_final', dest='min_frac_final', metavar='FLOAT', type=float, default=0.76,
                   help='final minimum fraction of supporting reads per strand (default: 0.76)')
parser.add_argument('-O', '--min_hamming_dist', dest='min_hamming_dist', metavar='INT', type=int, default=0,
                   help='minimum hamming distance between k-mer starting at the fragment position and any other k-mer \
                   in the genome (default: 0, i.e. nothing filtered)')
parser.add_argument('-P', '--max_cov_percentile', dest='max_cov_percentile', metavar='FLOAT', type=float, default=1,
                   help='maximum sequencing coverage percentile of the variant position (default: 1, i.e. nothing filtered)')
parser.add_argument('-Q', '--require_overlap', dest='require_overlap', action='store_true',
                   help='whether to require support from both forward and reverse mapping reads, i.e. variant within \
                   the read1-read2 overlap (default: do not require)')

try: # run this if in a jupyter notebook
    get_ipython()
    print('Determined code is running in Jupyter')
    if os.getcwd()[:8] != '/scratch': # switch to the scratch directory where all the data files are
        os.chdir(f'/scratch/cam02551/{os.getcwd().split("/")[-2]}')
    args = parser.parse_args('-i tmp/test_add_duplex.tsv'.split()) # used for testing
except: # run this if in a terminal
    args = parser.parse_args()

if args.output and '/' in args.output:
    os.makedirs(os.path.dirname(args.output), exist_ok=True)
sys.stderr.write('Running filter_duplex_variants.py with arguments:\n' + '\n'.join([f'{key}={val}' for key, val in vars(args).items()]) + '\n')

In [ ]:
# returns a boolean list of whether each variant in the df passes filter G
def passes_g(df):
    G = []
    for r in df.itertuples():
        if r.ref != '*' and r.alt != '*':
            g = True
        elif r.alt == '*' and r.pos > r.frag_start + r.frag_len // 2: # if a deletion nearer the end of the fragment
            g = r.frag_start + r.frag_len - r.pos - len(r.alt) >= args.min_indel_end_dist # count distance from the last base of the deletion
        else:
            g = r.pos - r.frag_start >= args.min_indel_end_dist and r.frag_start + r.frag_len - r.pos - 1 >= args.min_indel_end_dist
        G.append(g)
    return G

In [ ]:
# Returs a boolean list of whether each variant in the df passes filter M
# Variants fail this filter if their fragment has 2 variants >max_dist bp apart
# max_dist=0 never allows more than one variant in a fragment, max_dist=1 allows a pair of back-to-back variants
# input df must be sorted by chrom, frag_start, frag_len, frag_umi, pos
def passes_m(df):
    df.frag_umi = df.frag_umi.fillna('') # need to set NAs to something else, as NA != NA
    rows = df.itertuples()
    prev_frag = ('')
    keep = []
    poss = []
    while True:
        r = next(rows, None)
        if r is None or (r.chrom, r.frag_start, r.frag_len, r.frag_umi) != prev_frag:
            keep += [poss[-1] - poss[0] <= args.max_contamination_dist] * len(poss) if len(poss) > 0 else []
            if r is None:
                return keep
            prev_frag = (r.chrom, r.frag_start, r.frag_len, r.frag_umi)
            poss = []
        poss.append(r.pos)

In [ ]:
# load the variants
sys.stderr.write('loading unfiltered variants\n')
if args.input is not None:
    df = pd.read_table(args.input, quoting=csv.QUOTE_NONE)
else:
    df = pd.read_table(sys.stdin, quoting=csv.QUOTE_NONE)
sys.stderr.write(f'loaded {len(df)} unfiltered variants\n')

In [ ]:
# add columns for whether each filter is passed
df['A'] = (df.f_read1_sup / df.f_read1_cov >= args.min_frac_init) & (df.r_read1_sup / df.r_read1_cov >= args.min_frac_init)
df['B'] = (df.f_read1_cov >= args.min_strand_cov) & (df.r_read1_cov >= args.min_strand_cov)
df['C'] = (df.f_read1_cov + df.r_read1_cov >= args.min_total_cov)
df['D'] = (df.high_sup >= args.min_high_bq)
df['E'] = (df.avg_mq >= args.min_avg_mq)
df['F'] = (df.end_mismatch_rate <= args.max_end_mismatch)
df['G'] = passes_g(df)
df['H'] = (df.cov_per >= args.min_cov_percentile)
df['I'] = (df.poly_at <= args.max_poly_at)
df['J'] = (df.poly_rep <= args.max_poly_rep)
df['K'] = (df.control_sup <= args.max_control_sup)
df['L'] = (df.frag_len <= args.max_frag_len)
# skip M for now because it requires a slow sort
df['N'] = (df.f_read1_sup / df.f_read1_cov >= args.min_frac_final) & (df.r_read1_sup / df.r_read1_cov >= args.min_frac_final)
df['O'] = (df.dup_dist >= args.min_hamming_dist)
df['P'] = (df.cov_per <= args.max_cov_percentile)
df['Q'] = ((df.f_sup > 0) & (df.r_sup > 0)) if args.require_overlap else True

sys.stderr.write(f'filter A: {sum(~df.A)} variants fail and {sum(~df.A & df.B & df.C & df.D & df.E & df.F & df.G & df.H & df.I & df.J & df.K & df.L & df.N & df.O & df.P & df.Q)} variants fail only this filter (and potentially M)\n')
sys.stderr.write(f'filter B: {sum(~df.B)} variants fail and {sum(df.A & ~df.B & df.C & df.D & df.E & df.F & df.G & df.H & df.I & df.J & df.K & df.L & df.N & df.O & df.P & df.Q)} variants fail only this filter (and potentially M)\n')
sys.stderr.write(f'filter C: {sum(~df.C)} variants fail and {sum(df.A & df.B & ~df.C & df.D & df.E & df.F & df.G & df.H & df.I & df.J & df.K & df.L & df.N & df.O & df.P & df.Q)} variants fail only this filter (and potentially M)\n')
sys.stderr.write(f'filter D: {sum(~df.D)} variants fail and {sum(df.A & df.B & df.C & ~df.D & df.E & df.F & df.G & df.H & df.I & df.J & df.K & df.L & df.N & df.O & df.P & df.Q)} variants fail only this filter (and potentially M)\n')
sys.stderr.write(f'filter E: {sum(~df.E)} variants fail and {sum(df.A & df.B & df.C & df.D & ~df.E & df.F & df.G & df.H & df.I & df.J & df.K & df.L & df.N & df.O & df.P & df.Q)} variants fail only this filter (and potentially M)\n')
sys.stderr.write(f'filter F: {sum(~df.F)} variants fail and {sum(df.A & df.B & df.C & df.D & df.E & ~df.F & df.G & df.H & df.I & df.J & df.K & df.L & df.N & df.O & df.P & df.Q)} variants fail only this filter (and potentially M)\n')
sys.stderr.write(f'filter G: {sum(~df.G)} variants fail and {sum(df.A & df.B & df.C & df.D & df.E & df.F & ~df.G & df.H & df.I & df.J & df.K & df.L & df.N & df.O & df.P & df.Q)} variants fail only this filter (and potentially M)\n')
sys.stderr.write(f'filter H: {sum(~df.H)} variants fail and {sum(df.A & df.B & df.C & df.D & df.E & df.F & df.G & ~df.H & df.I & df.J & df.K & df.L & df.N & df.O & df.P & df.Q)} variants fail only this filter (and potentially M)\n')
sys.stderr.write(f'filter I: {sum(~df.I)} variants fail and {sum(df.A & df.B & df.C & df.D & df.E & df.F & df.G & df.H & ~df.I & df.J & df.K & df.L & df.N & df.O & df.P & df.Q)} variants fail only this filter (and potentially M)\n')
sys.stderr.write(f'filter J: {sum(~df.J)} variants fail and {sum(df.A & df.B & df.C & df.D & df.E & df.F & df.G & df.H & df.I & ~df.J & df.K & df.L & df.N & df.O & df.P & df.Q)} variants fail only this filter (and potentially M)\n')
sys.stderr.write(f'filter K: {sum(~df.K)} variants fail and {sum(df.A & df.B & df.C & df.D & df.E & df.F & df.G & df.H & df.I & df.J & ~df.K & df.L & df.N & df.O & df.P & df.Q)} variants fail only this filter (and potentially M)\n')
sys.stderr.write(f'filter L: {sum(~df.L)} variants fail and {sum(df.A & df.B & df.C & df.D & df.E & df.F & df.G & df.H & df.I & df.J & df.K & ~df.L & df.N & df.O & df.P & df.Q)} variants fail only this filter (and potentially M)\n')
sys.stderr.write(f'filter N: {sum(~df.N)} variants fail and {sum(df.A & df.B & df.C & df.D & df.E & df.F & df.G & df.H & df.I & df.J & df.K & df.L & ~df.N & df.O & df.P & df.Q)} variants fail only this filter (and potentially M)\n')
sys.stderr.write(f'filter O: {sum(~df.O)} variants fail and {sum(df.A & df.B & df.C & df.D & df.E & df.F & df.G & df.H & df.I & df.J & df.K & df.L & df.N & ~df.O & df.P & df.Q)} variants fail only this filter (and potentially M)\n')
sys.stderr.write(f'filter P: {sum(~df.P)} variants fail and {sum(df.A & df.B & df.C & df.D & df.E & df.F & df.G & df.H & df.I & df.J & df.K & df.L & df.N & df.O & ~df.P & df.Q)} variants fail only this filter (and potentially M)\n')
sys.stderr.write(f'filter Q: {sum(~df.Q)} variants fail and {sum(df.A & df.B & df.C & df.D & df.E & df.F & df.G & df.H & df.I & df.J & df.K & df.L & df.N & df.O & df.P & ~df.Q)} variants fail only this filter (and potentially M)\n')

sys.stderr.write('executing filters A through L\n')
df = df[df.A & df.B & df.C & df.D & df.E & df.F & df.G & df.H & df.I & df.J & df.K & df.L]
sys.stderr.write(f'{len(df)} variants remain\n')


In [ ]:
# add a column for filter M
df = df.sort_values('chrom frag_start frag_len frag_umi pos ref alt'.split())
df['M'] = passes_m(df)
sys.stderr.write(f'filter M: {sum(~df.M)} of remaining variants fail and {sum(df.A & df.B & df.C & df.D & df.E & df.F & df.G & df.H & df.I & df.J & df.K & df.L & ~df.M & df.N & df.O & df.P & df.Q)} of remaining variants fail only this filter\n')

sys.stderr.write('executing filter M\n')
if len(df) > 0:
    df = df[df.M] # df would lose its columns if this were run on an empty df
sys.stderr.write(f'{len(df)} variants remain\n')

In [ ]:
sys.stderr.write('executing filters N through Q\n')
if len(df) > 0:
    df = df[df.N & df.O & df.P & df.Q]
sys.stderr.write(f'{len(df)} variants remain\n')

In [ ]:
# delete filter columns
for fil in 'A B C D E F G H I J K L M N O P Q'.split():
    del df[fil]

In [ ]:
# write output
if args.output is not None:
    df.to_csv(args.output, index=False, sep='\t')
else:
    df.to_csv(sys.stdout, index=False, sep='\t')

In [ ]:
sys.stderr.write(f'completed filter_duplex_variants.py on {args.input}\n')